## Fine-Tuning and Evaluating LLMs with SageMaker Pipelines and MLflow

Running hundreds of experiments, comparing the results, and keeping a track of the ML lifecycle can become very complex. This is where MLflow can help streamline the ML lifecycle, from data preparation to model deployment. By integrating MLflow into your LLM workflow, you can efficiently manage experiment tracking, model versioning, and deployment, providing reproducibility. With MLflow, you can track and compare the performance of multiple LLM experiments, identify the best-performing models, and deploy them to production environments with confidence. 

You can create workflows with SageMaker Pipelines that enable you to prepare data, fine-tune models, and evaluate model performance with simple Python code for each step. 

Now you can use SageMaker managed MLflow to run LLM fine-tuning and evaluation experiments at scale. Specifically:

- MLflow can manage tracking of fine-tuning experiments, comparing evaluation results of different runs, model versioning, deployment, and configuration (such as data and hyperparameters)
- SageMaker Pipelines can orchestrate multiple experiments based on the experiment configuration 
  

The following figure shows the overview of the solution.
![](./ml-16670-arch-with-mlflow.png)

## Prerequisites 
Before you begin, make sure you have the following prerequisites in place:

- [HuggingFace access token](https://huggingface.co/docs/hub/en/security-tokens) – You need a HuggingFace login token to access the DeepSeek-R1-Distill-Llama-8B model and datasets used in this post.

- The notebook will download the DeepSeek-R1-Distill-Llama-8B model from HuggingFace and upload it to your S3 bucket for fine-tuning.

### 1. Setup and Dependencies
Restart the kernel after executing below cells

In [41]:
%pip install -r ./scripts/requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [42]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Importing Libraries and Setting Up Environment**

This part imports all necessary Python modules. It includes SageMaker-specific imports for pipeline creation and execution, which will be used to define the pipeline steps.

In [1]:
import os
import boto3
import sagemaker
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.function_step import step
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.steps import CacheConfig
from botocore.exceptions import ClientError

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### 2. SageMaker Session and IAM Role

`get_execution_role()`: Retrieves the IAM role that SageMaker will use to access AWS resources. This role needs appropriate permissions for tasks like accessing S3 buckets and creating SageMaker resources.

In [2]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
role = "arn:aws:iam::329542461890:role/data-scientist-role"
instance_type = "ml.m5.xlarge"
pipeline_name = "AIM405-deepseek-finetune-pipeline"
bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix
if default_prefix:
    input_path = f'{default_prefix}/datasets/llm-fine-tuning-modeltrainer-sft'
else:
    input_path = f'datasets/llm-fine-tuning-modeltrainer-sft'

model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
model_id_filesafe = model_id.replace("/","_").replace(".", "_")

### 3. Configuration

MLflow integration is crucial for experiment tracking and management. **Update the ARN for the MLflow tracking server.**

mlflow_arn: The ARN for the MLflow tracking server. You can get this ARN from SageMaker Studio UI. This allows the pipeline to log metrics, parameters, and artifacts to a central location.

This example requires a SageMaker with MLflow tracking server to track experiments and manage model artifacts. To create your own tracking server please refer to the [SageMaker documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html). Once you have created your tracking server, please copy the tracking server ARN to the `mlflow_tracking server_arn` variable in the cell below.

In [3]:
mlflow_tracking_server_arn = "arn:aws:sagemaker:us-east-1:329542461890:mlflow-tracking-server/my-tracking-server"

if not mlflow_tracking_server_arn:
    try:
        response = boto3.client('sagemaker').describe_mlflow_tracking_server(
            TrackingServerName='genai-mlflow-tracker'
        )
        mlflow_tracking_server_arn = response['TrackingServerArn']
        print(f"MLflow Tracking Server ARN: {mlflow_tracking_server_arn}")
    except ClientError:
        print("No MLflow Tracking Server Found, please input a value for mlflow_tracking_server_arn")

os.environ["mlflow_tracking_server_arn"] = mlflow_tracking_server_arn
os.environ["pipeline_name"] = pipeline_name

In [4]:
%%writefile config.yaml
SchemaVersion: '1.0'
SageMaker:
  PythonSDK:
    Modules:
      RemoteFunction:
        # role arn is not required if in SageMaker Notebook instance or SageMaker Studio
        # Uncomment the following line and replace with the right execution role if in a local IDE
        # RoleArn: <replace the role arn here>
        InstanceType: ml.m5.xlarge
        Dependencies: ./scripts/requirements.txt
        IncludeLocalWorkDir: true
        CustomFileFilter:
          IgnoreNamePatterns: # files or directories to ignore
          - "*.ipynb" # all notebook files

Overwriting config.yaml


In [5]:
# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

### 4. Download Model Data from Huggingface

In [6]:
from huggingface_hub import snapshot_download


# Simple function to check if file exists in S3
def s3_file_exists(s3_client, bucket, key):
    try:
        s3_client.head_object(Bucket=bucket, Key=key)
        return True
    except ClientError:
        return False

# Simple S3 upload function that checks if files exist before uploading
def simple_s3_upload(local_dir, s3_bucket, s3_prefix, skip_existing=True):
    """
    Upload files to S3, skipping files that already exist.
    
    Args:
        local_dir (str): Local directory containing files to upload
        s3_bucket (str): S3 bucket name
        s3_prefix (str): S3 prefix (folder path)
        skip_existing (bool): Whether to skip files that already exist in S3
        
    Returns:
        tuple: (uploaded_files, skipped_files, failed_files)
    """
    s3_client = boto3.client('s3')
    uploaded_files = []
    skipped_files = []
    failed_files = []
    
    # Get all local files
    local_files = []
    for root, _, files in os.walk(local_dir):
        for filename in files:
            local_path = os.path.join(root, filename)
            rel_path = os.path.relpath(local_path, local_dir)
            s3_key = os.path.join(s3_prefix, rel_path).replace('\\', '/')
            local_files.append((local_path, s3_key))
    
    print(f"Found {len(local_files)} files in {local_dir}")
    
    # Process each file sequentially
    for local_path, s3_key in local_files:
        try:
            # Check if file exists in S3
            if skip_existing and s3_file_exists(s3_client, s3_bucket, s3_key):
                print(f"Skipping {s3_key} (file exists in S3)")
                skipped_files.append(s3_key)
                continue
            
            # Upload the file
            print(f"Uploading {local_path} to s3://{s3_bucket}/{s3_key}")
            s3_client.upload_file(
                local_path, 
                s3_bucket, 
                s3_key,
                ExtraArgs={'ACL': 'bucket-owner-full-control'}
            )
            uploaded_files.append(s3_key)
            
        except Exception as e:
            print(f"Failed to upload {local_path}: {str(e)}")
            failed_files.append((s3_key, str(e)))
    
    print(f"\nUpload Summary:")
    print(f"  - Uploaded: {len(uploaded_files)} files")
    print(f"  - Skipped: {len(skipped_files)} files")
    print(f"  - Failed: {len(failed_files)} files")
    
    return uploaded_files, skipped_files, failed_files

# Set local and S3 model paths
model_local_location = f"../models/{model_id_filesafe}"
if default_prefix:
    model_s3_destination = f"s3://{bucket_name}/{default_prefix}/models/{model_id_filesafe}"
    prefix = f"{default_prefix}/models/{model_id_filesafe}"
else:
    model_s3_destination = f"s3://{bucket_name}/models/{model_id_filesafe}"
    prefix = f"models/{model_id_filesafe}"

print("Downloading model ", model_id)
os.makedirs(model_local_location, exist_ok=True)

try:
    snapshot_download(repo_id=model_id, local_dir=model_local_location)
    print(f"Model {model_id} downloaded under {model_local_location}")
    
    print(f"Beginning Model Upload to {model_s3_destination}...")
    
    # Use the simple upload function without threads or batch processing
    uploaded, skipped, failed = simple_s3_upload(
        local_dir=model_local_location,
        s3_bucket=bucket_name,
        s3_prefix=prefix,
        skip_existing=True
    )
 
    print(f"Model successfully uploaded to: \n {model_s3_destination}")
except Exception as e:
    print(f"Error during model download or upload: {e}")
    raise

os.environ["model_location"] = model_s3_destination

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Model deepseek-ai/DeepSeek-R1-Distill-Llama-8B downloaded under ../models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B
Beginning Model Upload to s3://sagemaker-us-east-1-329542461890/models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B...
Found 34 files in ../models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/generation_config.json (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/model.safetensors.index.json (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/README.md (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/config.json (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/tokenizer_config.json (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/.gitattributes (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/LICENSE (file exists in S3)
Skipping models/deepseek-ai_DeepSeek-R1-Dis

### 5. Configure Fine-Tuning Job

This section defines the core components of the SageMaker pipeline.

**Training Configuration**

The train_config dictionary is comprehensive, including:

Experiment naming for tracking purposes
Model specifications (ID, version, name)
Infrastructure details (instance types and counts for fine-tuning and deployment)
Training hyperparameters (epochs, batch size)

This configuration allows for easy adjustment of the training process without changing the core pipeline code.

**LoRA Parameters**

Low-Rank Adaptation (LoRA) is an efficient fine-tuning technique that reduces the number of trainable parameters by adding low-rank decomposition matrices to existing weights rather than updating all model weights. This significantly reduces memory requirements and training time while maintaining performance comparable to full fine-tuning.

In [7]:
%%bash

cat > ./args.yaml <<EOF

# MLflow Config
mlflow_uri: "${mlflow_tracking_server_arn}"                # The URI for the MLflow tracking server 
mlflow_experiment_name: "${pipeline_name}"  # Name of the MLflow experiment for organizing runs


model_id: "${model_location}"              # Hugging Face model id, or S3 location of base model

# SageMaker specific parameters 
output_dir: "/opt/ml/model"                # Path where SageMaker will upload the model 
train_dataset_path: "/opt/ml/input/data/train/"   # Path where FSx saves train dataset
test_dataset_path: "/opt/ml/input/data/test/"     # Path where FSx saves test dataset

# Training parameters
max_seq_length: 1500                       # Maximum sequence length for inputs (affects memory usage)
                                           # Higher values allow for longer context but require more memory
                                           # Range: 512-4096 depending on model architecture and hardware

# LoRA parameters (Low-Rank Adaptation)
lora_r: 8                                  # Rank of the LoRA update matrices
                                           # Lower values (4-16) are more efficient, higher values (32-64) can improve quality
                                           # Recommended range: 8-64 depending on task complexity
lora_alpha: 16                             # Scaling factor for the LoRA update
                                           # Generally set to 2x lora_r for good performance
lora_dropout: 0.1                          # Dropout probability for LoRA layers
                                           # Range: 0.0-0.5, helps prevent overfitting

# Optimizer parameters
learning_rate: 2e-4                        # Learning rate for parameter updates
                                           # Range: 1e-5 to 5e-4 for LoRA fine-tuning
                                           # Too high: training instability, too low: slow convergence

# Training loop parameters
num_train_epochs: 1                        # Number of complete passes through the training dataset
                                           # More epochs can improve performance but risk overfitting
                                           # Range: 1-5 for LoRA fine-tuning
per_device_train_batch_size: 2             # Number of samples per GPU during training
                                           # Larger values improve training speed but require more memory
                                           # Range: 1-8 for large models on common GPUs
per_device_eval_batch_size: 1              # Number of samples per GPU during evaluation
                                           # Can typically be larger than training batch size
gradient_accumulation_steps: 2             # Accumulate gradients over multiple steps
                                           # Effectively increases batch size by this factor
                                           # Useful when limited by GPU memory

# Memory optimization techniques
gradient_checkpointing: true               # Reduces memory usage by recomputing activations during backward pass
                                           # Trades computation for memory, ~20% slower but enables larger models/sequences
fp16: true                                 # Use half-precision floating point (speeds up training, reduces memory)
bf16: false                                # Use bfloat16 precision (better numerical stability than fp16)
                                           # Also enables FlashAttention2 (requires Ampere/Hopper GPU+ eg:A10, A100, H100)
tf32: false                                # Use TensorFloat-32 precision (NVIDIA Ampere+ GPUs only)

#uncomment here for fsdp - start
# fsdp: "full_shard auto_wrap offload"     # Fully Sharded Data Parallel training
                                           # Splits model states across multiple GPUs
# fsdp_config:                             # Configuration for FSDP
#     backward_prefetch: "backward_pre"    # Prefetches parameters before backward pass
#     cpu_ram_efficient_loading: true      # More memory-efficient parameter loading
#     offload_params: true                 # Offloads parameters to CPU when not in use
#     forward_prefetch: false              # Don't prefetch parameters for forward pass
#     use_orig_params: true                # Use original parameter ordering
#uncomment here for fsdp - end

merge_weights: true                        # Merge adapter weights into the base model
                                           # true: produces standalone model, false: keeps adapter separate
EOF

In [8]:
from sagemaker.s3 import S3Uploader

if default_prefix:
    input_path = f"s3://{bucket_name}/{default_prefix}/training_config/{model_id_filesafe}"
else:
    input_path = f"s3://{bucket_name}/training_config/{model_id_filesafe}"

# upload the model yaml file to s3
model_yaml = "args.yaml"
train_config_s3_path = S3Uploader.upload(local_path=model_yaml, desired_s3_uri=f"{input_path}/config")

print(f"Training config uploaded to:")
print(train_config_s3_path)

sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/generative-ai-on-amazon-sagemaker/workshops/fine-tuning-with-sagemakerai-and-bedrock/task_05_fmops
Training config uploaded to:
s3://sagemaker-us-east-1-329542461890/training_config/deepseek-ai_DeepSeek-R1-Distill-Llama-8B/config/args.yaml


### 6. Pipeline Creation and Execution

This final section brings all the components together into an executable pipeline.

**Creating the Pipeline**

The pipeline object is created with all defined steps.

In [9]:
from steps import (
    preprocess_step,
    finetune_step,
    deploy_step,
    quantitative_eval_step,
    qualitative_eval_step,
    model_registration_step
)
from sagemaker.workflow.step_collections import StepCollection

preprocessing_step = preprocess_step.preprocess(
    tracking_server_arn=mlflow_tracking_server_arn,
    experiment_name=pipeline_name,
    run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
    input_path=input_path
)

training_step = finetune_step.train(
    tracking_server_arn=mlflow_tracking_server_arn,
    experiment_name=pipeline_name,
    run_id=preprocessing_step[0],
    train_dataset_s3_path=preprocessing_step[1],
    test_dataset_s3_path=preprocessing_step[2],
    train_config_s3_path=train_config_s3_path,
    role=role,
    model_id=model_s3_destination,
)
run_id=training_step[0]
model_artifacts_s3_path=training_step[2]
output_path=training_step[3]

deploy_step = deploy_step.deploy(
    model_artifacts_s3_path=model_artifacts_s3_path,
    output_path=output_path,
    model_id=model_s3_destination,
)
endpoint_name=deploy_step

quantitative_eval_step = quantitative_eval_step.quantitative_evaluate(
    tracking_server_arn=mlflow_tracking_server_arn,
    experiment_name=pipeline_name,
    run_id=run_id,
    endpoint_name=endpoint_name
)

qualitative_eval_step = qualitative_eval_step.qualitative_evaluate(
    tracking_server_arn=mlflow_tracking_server_arn,
    experiment_name=pipeline_name,
    run_id=run_id,
    endpoint_name=endpoint_name
)

evaluation_gate = ConditionStep(
    name="EvaluationGate",
    depends_on=[qualitative_eval_step],
    conditions=[
        ConditionGreaterThanOrEqualTo(
            left=quantitative_eval_step["rougeL_f"],
            right=0.2
        ),
        ConditionGreaterThanOrEqualTo(
            left=qualitative_eval_step["avg_medical_accuracy"],
            right=3.0
        )
    ],
    if_steps=[
        model_registration_step.register_model(
            tracking_server_arn=mlflow_tracking_server_arn,
            experiment_name=pipeline_name,
            run_id=run_id,  # Assuming training_step returns run_id as first output
            model_artifacts_s3_path=model_artifacts_s3_path,  # Assuming training_step returns artifacts path as second output
            model_id=model_id,
            model_name=f"Fine-Tuned-Medical-DeepSeek",
            endpoint_name=endpoint_name,
            evaluation_score=quantitative_eval_step["rougeL_f"],  # Get the evaluation score
            pipeline_name=pipeline_name,
            model_description="Fine-tuned medical LLM for clinical reasoning and diagnostics"
        )
    ],
    else_steps=[
        FailStep(
            name="EvaluationFailed",
            error_message="Model evaluation failed to meet quality thresholds."
        )
    ]
)

# Combining the steps into the pipeline definition
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        instance_type,
    ],
    steps=[
        preprocessing_step,
        training_step,
        deploy_step,
        quantitative_eval_step,
        evaluation_gate
    ],
)

INFO:datasets:PyTorch version 2.6.0 available.
INFO:datasets:TensorFlow version 2.18.0 available.


**Upserting the Pipeline**

This step either creates a new pipeline in SageMaker or updates an existing one with the same name. It's a key part of the MLOps process, allowing for iterative refinement of the pipeline.

In [10]:
pipeline.upsert(role)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-09-15 19:40:19,366 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/DataPreprocessing/2025-09-15-19-40-17-829/function
2025-09-15 19:40:19,431 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/DataPreprocessing/2025-09-15-19-40-17-829/arguments
2025-09-15 19:40:19,620 sagemaker.remote_function INFO     Copied dependencies file at './scripts/requirements.txt' to '/tmp/tmpfb7vb7bb/requirements.txt'
2025-09-15 19:40:19,655 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/DataPreprocessing/2025-09-15-19-40-17-829/pre_exec_script_and_dependencies'
2025-09-15 19:40:19,663 sagemaker.remote_function INFO     Copied user workspace to '/tmp/tmprcsjitzu/temp_workspace/sagemaker_

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-09-15 19:40:21,013 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/ModelFineTuning/2025-09-15-19-40-17-829/function
2025-09-15 19:40:21,107 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/ModelFineTuning/2025-09-15-19-40-17-829/arguments
2025-09-15 19:40:21,179 sagemaker.remote_function INFO     Copied dependencies file at './scripts/requirements.txt' to '/tmp/tmppb5hw8pw/requirements.txt'
2025-09-15 19:40:21,207 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/ModelFineTuning/2025-09-15-19-40-17-829/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


...

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-09-15 19:40:25,566 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/ModelRegistration/2025-09-15-19-40-17-829/function
2025-09-15 19:40:25,628 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/ModelRegistration/2025-09-15-19-40-17-829/arguments
2025-09-15 19:40:25,682 sagemaker.remote_function INFO     Copied dependencies file at './scripts/requirements.txt' to '/tmp/tmpjbq8x0jf/requirements.txt'
2025-09-15 19:40:25,715 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/ModelRegistration/2025-09-15-19-40-17-829/pre_exec_script_and_dependencies'


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.IncludeLocalWorkDir
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.CustomFileFilter.IgnoreNamePatterns


2025-09-15 19:40:27,000 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/QualitativeModelEvaluation/2025-09-15-19-40-17-829/function
2025-09-15 19:40:27,068 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/QualitativeModelEvaluation/2025-09-15-19-40-17-829/arguments
2025-09-15 19:40:27,156 sagemaker.remote_function INFO     Copied dependencies file at './eval/requirements.txt' to '/tmp/tmprwbp1rt0/requirements.txt'
2025-09-15 19:40:27,184 sagemaker.remote_function INFO     Successfully uploaded dependencies and pre execution scripts to 's3://sagemaker-us-east-1-329542461890/AIM405-deepseek-finetune-pipeline/QualitativeModelEvaluation/2025-09-15-19-40-17-829/pre_exec_script_and_dependencies'
2025-09-15 19:40:27,613 sagemaker.remote_function INFO     Uploading serialized function code to s3://s

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:329542461890:pipeline/AIM405-deepseek-finetune-pipeline',
 'PipelineVersionId': 38,
 'ResponseMetadata': {'RequestId': '4edfb27f-a58f-4b54-b689-80c5af37636f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4edfb27f-a58f-4b54-b689-80c5af37636f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Mon, 15 Sep 2025 19:40:29 GMT'},
  'RetryAttempts': 0}}

**Starting the Pipeline Execution**

This command kicks off the actual execution of the pipeline in SageMaker. From this point, SageMaker will orchestrate the execution of each step, managing resources and data flow between steps.

In [11]:
execution = pipeline.start()

In [21]:
register_model(
    tracking_server_arn="arn:aws:sagemaker:us-east-1:329542461890:mlflow-tracking-server/my-tracking-server",
    experiment_name="AIM405-deepseek-finetune-pipeline",
    run_id="ac48b182ac8746178c9dc26c42176a2c",  # Assuming training_step returns run_id as first output
    model_artifacts_s3_path="s3://sagemaker-us-east-1-329542461890/deepseek-finetune-2025-09-15-18-39-44-2025-09-15-18-39-44-661/output/model.tar.gz",  # Assuming training_step returns artifacts path as second output
    model_id="s3://sagemaker-us-east-1-329542461890/models/deepseek-ai_DeepSeek-R1-Distill-Llama-8B",
    model_name=f"Fine-Tuned-Medical-DeepSeek",
    endpoint_name="deepseek-ai-DeepSeek-R1-Distill-Llama-8B-sft-djl",
    evaluation_score=0.22410036842493342,  # Get the evaluation score
    pipeline_name="AIM405-deepseek-finetune-pipeline",
    model_description="Fine-tuned medical LLM for clinical reasoning and diagnostics"
)

Registering model: Fine-Tuned-Medical-DeepSeek
🏃 View run pewfpvcn2tde at: https://us-east-1.experiments.sagemaker.aws/#/experiments/1/runs/ac48b182ac8746178c9dc26c42176a2c
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/1


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/mlflow/tracking/_tracking_service/client.py:401 in       │
│ log_param                                                                                        │
│                                                                                                  │
│   398 │   │   param = Param(key, str(value))                                                     │
│   399 │   │   try:                                                                               │
│   400 │   │   │   if synchronous:                                                                │
│ ❱ 401 │   │   │   │   self.store.log_param(run_id, param)                                        │
│   402 │   │   │   │   return value                                                               │
│   403 │   │   │   else:                                                                          │
│   404 │   │   │   │   return self.store.log_param_async(run_id, param)                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/mlflow/store/tracking/rest_store.py:644 in log_param     │
│                                                                                                  │
│    641 │   │   req_body = message_to_json(                                                       │
│    642 │   │   │   LogParam(run_uuid=run_id, run_id=run_id, key=param.key, value=param.value)    │
│    643 │   │   )                                                                                 │
│ ❱  644 │   │   self._call_endpoint(LogParam, req_body)                                           │
│    645 │                                                                                         │
│    646 │   def set_experiment_tag(self, experiment_id, tag):                                     │
│    647 │   │   """                                                                               │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/mlflow/store/tracking/rest_store.py:134 in               │
│ _call_endpoint                                                                                   │
│                                                                                                  │
│    131 │   │   else:                                                                             │
│    132 │   │   │   endpoint, method = _METHOD_TO_INFO[api]                                       │
│    133 │   │   response_proto = api.Response()                                                   │
│ ❱  134 │   │   return call_endpoint(                                                             │
│    135 │   │   │   self.get_host_creds(),                                                        │
│    136 │   │   │   endpoint,                                                                     │
│    137 │   │   │   method,                                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/mlflow/utils/rest_utils.py:554 in call_endpoint          │
│                                                                                                  │
│   551 │   │   call_kwargs["json"] = json_body                                                    │
│   552 │   │   response = http_request(**call_kwargs)                                             │
│   553 │                                                                                          │
│ ❱ 554 │   response = verify_rest_response(response, endpoint)                                    │
│   555 │   response_to_parse = response.text                

In [19]:
def register_model(
    tracking_server_arn: str,
    experiment_name: str,
    run_id: str,
    model_artifacts_s3_path: str,
    model_id: str,
    model_name: str,
    endpoint_name: str,
    evaluation_score: float,
    pipeline_name: str,
    model_description: str
):
    import json
    import mlflow
    import boto3
    import os
    import tempfile
    import time
    from datetime import datetime
    
    print(f"Registering model: {model_name}")
    
    # Set up MLflow tracking
    mlflow.set_tracking_uri(tracking_server_arn)
    mlflow.set_experiment(experiment_name)
    
    # Connect to MLflow with the specific run
    with mlflow.start_run(run_id=run_id):
        # Create model metadata
        tags = {
            "model_id": model_id,
            "base_model": model_id.split('/')[-1],
            "task": "medical_qa",
            "framework": "pytorch",
            "endpoint_name": endpoint_name,
            "model_artifacts_s3_path": model_artifacts_s3_path,
            "deployment_timestamp": datetime.now().isoformat(),
            "description": model_description,
            "registered_by": pipeline_name
        }
            
        # Log model info as parameters
        mlflow.log_param("registered_model_name", model_name)
        mlflow.log_param("model_artifacts_path", model_artifacts_s3_path)
        mlflow.log_param("evaluation_score", evaluation_score)
        mlflow.log_param("endpoint_name", endpoint_name)
        # mlflow.log_param("registration_timestamp", datetime.now().isoformat())
        mlflow.log_param("registration_timestamp", "2025-09-15T19:28:36.049313")
        
        # Log endpoint information as an artifact
        model_info = {
            "model_name": model_name,
            "model_id": model_id,
            "endpoint_name": endpoint_name,
            "model_artifacts_s3_path": model_artifacts_s3_path,
            "evaluation_score": float(evaluation_score),
            # "registration_timestamp": datetime.now().isoformat()
            "registration_timestamp": "2025-09-15T19:28:36.049313"
        }
        
        with open("/tmp/model_info.json", "w") as f:
            json.dump(model_info, f, indent=2)
        mlflow.log_artifact("/tmp/model_info.json")
        
        # Create model card
        model_card = f"""
        # {model_name}
        
        ## Model Information
        - **Base Model**: {model_id}
        - **Task**: Medical Question Answering
        - **Evaluation Score**: {evaluation_score:.4f}
        - **Endpoint**: {endpoint_name}
        
        ## Description
        {model_description}
        
        ## Registration Details
        - Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        - Pipeline: {pipeline_name}
        """
        
        with open("/tmp/model_card.md", "w") as f:
            f.write(model_card)
        mlflow.log_artifact("/tmp/model_card.md")
        
        # PART 1: REGISTER WITH MLFLOW MODEL REGISTRY
        mlflow_version = None
        try:
            client = mlflow.tracking.MlflowClient()
            
            # Check if model exists and create if it doesn't
            try:
                client.get_registered_model(model_name)
                print(f"Model {model_name} already exists in MLflow registry")
            except mlflow.exceptions.MlflowException:
                client.create_registered_model(
                    name=model_name,
                    description=f"Fine-tuned medical LLM based on {model_id}"
                )
                print(f"Created new registered model: {model_name}")
            
            # Create empty model directory with artifacts
            with tempfile.TemporaryDirectory() as tmp_dir:
                # Create a minimal model file to log
                os.makedirs(os.path.join(tmp_dir, "model"), exist_ok=True)
                
                # Copy model info and card to directory
                with open(os.path.join(tmp_dir, "model", "model_info.json"), "w") as f:
                    json.dump(model_info, f, indent=2)
                    
                with open(os.path.join(tmp_dir, "model", "model_card.md"), "w") as f:
                    f.write(model_card)
                
                # Create a model reference file pointing to the S3 artifacts
                model_ref = {
                    "artifact_path": model_artifacts_s3_path,
                    "flavors": {
                        "pytorch": {
                            "model_data": model_artifacts_s3_path,
                            "pytorch_version": "2.0+"
                        }
                    },
                    "run_id": run_id,
                    "model_class": "LLM",
                    "model_format": "PyTorch"
                }
                
                with open(os.path.join(tmp_dir, "model", "MLmodel"), "w") as f:
                    json.dump(model_ref, f, indent=2)
                
                # Log artifacts directory as model
                mlflow.log_artifacts(tmp_dir, artifact_path="")
            
            # Now register the model - try both methods
            try:
                # Method 1: Use direct registration with source as run URI
                model_uri = f"runs:/{run_id}/model"
                model_details = mlflow.register_model(
                    model_uri=model_uri,
                    name=model_name,
                    tags=tags
                )
                mlflow_version = model_details.version
                
            except Exception as e1:
                print(f"Method 1 registration failed: {str(e1)}")
                
                try:
                    # Method 2: Create version with client API
                    model_version = client.create_model_version(
                        name=model_name,
                        source=f"runs:/{run_id}/model",  # Use run URI instead of direct S3
                        run_id=run_id,
                        description=f"Fine-tuned LLM deployed at endpoint: {endpoint_name}"
                    )
                    mlflow_version = model_version.version
                    
                    # Wait for model registration to complete
                    for _ in range(10):  # Try for up to ~50 seconds
                        version_details = client.get_model_version(model_name, model_version.version)
                        if version_details.status == "READY":
                            break
                        time.sleep(5)
                    
                    # Add tags to the registered model version
                    for key, value in tags.items():
                        client.set_model_version_tag(model_name, model_version.version, key, value)
                except Exception as e2:
                    print(f"Method 2 registration failed: {str(e2)}")
                    mlflow_version = "unknown"
            
            if mlflow_version and mlflow_version != "unknown":
                # Transition model to Production/Staging based on evaluation score
                if evaluation_score >= 0.3:  # Example threshold
                    client.transition_model_version_stage(
                        name=model_name,
                        version=mlflow_version,
                        stage="Production",
                        archive_existing_versions=True
                    )
                    print(f"Model {model_name} version {mlflow_version} promoted to Production")
                else:
                    client.transition_model_version_stage(
                        name=model_name,
                        version=mlflow_version,
                        stage="Staging",
                        archive_existing_versions=False
                    )
                    print(f"Model {model_name} version {mlflow_version} added to Staging due to lower evaluation score")
            
            print(f"Successfully registered model in MLflow: {model_name}, version: {mlflow_version}")
            
        except Exception as e:
            print(f"Error registering model in MLflow: {str(e)}")
            mlflow_version = "unknown"
            
        # PART 2: REGISTER WITH SAGEMAKER MODEL REGISTRY
        sm_model_version = "unknown"
        try:
            sm_client = boto3.client('sagemaker')
            
            # Create a normalized name for SageMaker resources
            sm_model_name = model_name.replace(".", "-").replace("_", "-")
            
            # Create or update model package group
            try:
                sm_client.describe_model_package_group(ModelPackageGroupName=sm_model_name)
                print(f"SageMaker model package group {sm_model_name} already exists")
            except sm_client.exceptions.ClientError:
                sm_client.create_model_package_group(
                    ModelPackageGroupName=sm_model_name,
                    ModelPackageGroupDescription=f"Fine-tuned LLM model: {model_name}"
                )
                print(f"Created SageMaker model package group: {sm_model_name}")
            
            # Create a model package and register it
            try:
                # Create model package
                response = sm_client.create_model_package(
                    ModelPackageGroupName=sm_model_name,
                    ModelPackageDescription=model_description,
                    SourceAlgorithmSpecification={
                        'SourceAlgorithms': [
                            {
                                'AlgorithmName': 'pytorch-llm',
                                'ModelDataUrl': model_artifacts_s3_path
                            }
                        ]
                    },
                    ValidationSpecification={
                        'ValidationRole': 'dummy-role',  # Required but not used
                        'ValidationProfiles': [
                            {
                                'ProfileName': 'ValidationProfile1',
                                'TransformJobDefinition': {
                                    'TransformInput': {
                                        'DataSource': {
                                            'S3DataSource': {
                                                'S3DataType': 'S3Prefix',
                                                'S3Uri': 's3://dummy-bucket/dummy-prefix'  # Required but not used
                                            }
                                        }
                                    },
                                    'TransformOutput': {
                                        'S3OutputPath': 's3://dummy-bucket/dummy-output'  # Required but not used
                                    },
                                    'TransformResources': {
                                        'InstanceType': 'ml.m5.large',  # Required but not used
                                        'InstanceCount': 1
                                    }
                                }
                            }
                        ]
                    },
                    ModelApprovalStatus='Approved',
                    MetadataProperties={
                        'GeneratedBy': pipeline_name,
                        'Repository': model_id,
                        'EvaluationScore': str(evaluation_score)
                    },
                    ModelMetrics={
                        'ModelQuality': {
                            'Statistics': {
                                'ContentType': 'application/json',
                                'S3Uri': f"s3://{model_artifacts_s3_path.split('/', 3)[2]}/{run_id}/artifacts/model_info.json"
                            }
                        }
                    }
                )
                
                sm_model_version = response['ModelPackageArn'].split('/')[-1]
                print(f"Created SageMaker model package: {sm_model_version}")
                
            except Exception as e_package:
                print(f"Error creating model package: {str(e_package)}")
            
            # Log SageMaker details
            mlflow.log_param("sagemaker_model_group", sm_model_name)
            mlflow.log_param("sagemaker_model_version", sm_model_version)
            
            print(f"Successfully integrated with SageMaker model registry")
            
        except Exception as e:
            print(f"Warning: Error in SageMaker model registry integration: {str(e)}")
            
    return model_name, str(mlflow_version)

# Clean up

In [ ]:
# Delete the endpoint to avoid incurring charges
import boto3
import time
import botocore

def delete_endpoint_with_retry(endpoint_name, max_retries=3, wait_seconds=10):
    """
    Delete a SageMaker endpoint with retry logic
    
    Args:
        endpoint_name (str): Name of the SageMaker endpoint to delete
        max_retries (int): Maximum number of retry attempts
        wait_seconds (int): Time to wait between retries in seconds
    
    Returns:
        bool: True if deletion was successful, False otherwise
    """
    sm_client = boto3.client('sagemaker')
    
    # First check if the endpoint exists
    try:
        sm_client.describe_endpoint(EndpointName=endpoint_name)
        endpoint_exists = True
    except sm_client.exceptions.ClientError as e:
        if "Could not find endpoint" in str(e):
            print(f"Endpoint {endpoint_name} does not exist, no cleanup needed.")
            return True
        else:
            print(f"Error checking endpoint existence: {e}")
            return False
    
    # If we get here, the endpoint exists and we should delete it
    for attempt in range(max_retries):
        try:
            print(f"Attempting to delete endpoint {endpoint_name} (attempt {attempt + 1}/{max_retries})")
            sm_client.delete_endpoint(EndpointName=endpoint_name)
            print(f"Endpoint {endpoint_name} deletion initiated successfully")
            
            # Wait for endpoint to be fully deleted
            print("Waiting for endpoint to be fully deleted...")
            
            # Poll until endpoint is deleted or max wait time is reached
            total_wait_time = 0
            max_wait_time = 300  # 5 minutes maximum wait
            while total_wait_time < max_wait_time:
                try:
                    sm_client.describe_endpoint(EndpointName=endpoint_name)
                    print(f"Endpoint still exists, waiting {wait_seconds} seconds...")
                    time.sleep(wait_seconds)
                    total_wait_time += wait_seconds
                except sm_client.exceptions.ClientError:
                    print(f"Endpoint {endpoint_name} successfully deleted")
                    return True
            
            # If we get here, the endpoint still exists after max_wait_time
            print(f"Warning: Endpoint deletion initiated but still exists after {max_wait_time} seconds")
            return False
            
        except botocore.exceptions.ClientError as e:
            if "ResourceInUse" in str(e) or "ResourceNotFound" in str(e):
                print(f"Error deleting endpoint: {e}")
                print(f"Retrying in {wait_seconds} seconds...")
                time.sleep(wait_seconds)
            else:
                print(f"Unexpected error deleting endpoint: {e}")
                return False
    
    print(f"Failed to delete endpoint {endpoint_name} after {max_retries} attempts")
    return False

# Clean up endpoint
try:
    model_name_safe = model_id.split('/')[-1].replace('.', '-').replace('_', '-')
    endpoint_name = f"{model_name_safe}-sft-djl"
    
    print(f"Cleaning up endpoint: {endpoint_name}")
    if delete_endpoint_with_retry(endpoint_name):
        print("Cleanup completed successfully")
    else:
        print("Warning: Endpoint cleanup may have failed, please check the SageMaker console")
        
except Exception as e:
    print(f"Error during endpoint cleanup: {str(e)}")
    print("You may need to manually delete the endpoint from the SageMaker console")